In [4]:
def get_data(endpoint, per_page = 100, cursor = None):
    url = "https://api.openalex.org/" + endpoint + "?per-page=" + per_page + "&cursor=" + cursor 
    data = requests.get(url)
    data_dict = json.loads(data.text)
    data_df = pd.DataFrame.from_dict(data_dict["results"])
    return data_df

In [27]:
import math
url = "https://api.openalex.org/sources?per-page=100&cursor=*"

sources = requests.get(url)
sources_dict = json.loads(sources.text)
total_pages = math.ceil(sources_dict["meta"]["count"]/100)
sources_df = pd.DataFrame.from_dict(sources_dict["results"])
sources_df.to_csv("sources.csv", mode='a', index=False)

while(total_pages > 1):
    cursor = sources_dict["meta"]["next_cursor"]
    sources_df = get_data("sources", "100", cursor)
    sources_df.to_csv("sources.csv", mode='a', index=False)
    total_pages -= 1

In [26]:
import math
url = "https://api.openalex.org/publishers?per-page=100&cursor=*"

publisher = requests.get(url)
publisher_dict = json.loads(publisher.text)
total_pages = math.ceil(publisher_dict["meta"]["count"]/100)
publisher_df = pd.DataFrame.from_dict(publisher_dict["results"])
publisher_df.to_csv("publisher.csv", mode='a', index=False)

# while(total_pages > 1):
#     cursor = publisher_dict["meta"]["next_cursor"]
#     publisher_df = get_data("publishers", "100", cursor)
#     publisher_df.to_csv("publisher.csv", mode='a', header=False, index=False)
#     total_pages -= 1

In [39]:
final_list_len = len(final_source)
result_list_len = len(ssci_in_sources)
# find percentage of journals in ssci_journals_in_sources
print("Percentage of journals in SSCI 2022 in OpenAlex: " + percentage(result_list_len, final_list_len))


Percentage of journals in SSCI 2022 in OpenAlex: 13.584905660377359%


In [42]:
print(percentage(ahci_in_sources_len + sce_in_sources_len + result_list_len, final_list_len))

80.75471698113208%


In [40]:
ahci_in_sources_len = len(ahci_in_sources)
print("Percentage of journals in AHCI 2022 in OpenAlex: " + percentage(ahci_in_sources_len, final_list_len))

Percentage of journals in AHCI 2022 in OpenAlex: 9.81132075471698%


In [41]:
sce_in_sources_len = len(sce_in_sources)
print("Percentage of journals in SCE 2022 in OpenAlex: " + percentage(sce_in_sources_len, final_list_len))

Percentage of journals in SCE 2022 in OpenAlex: 57.35849056603774%


In [1]:
import pandas as pd
import requests
import json 
import numpy as np

In [31]:
ssci_dfs = pd.read_excel("./SSCI2022.xlsx", sheet_name=None)
ahci_dfs = pd.read_excel("./AHCI2022.xlsx", sheet_name=None)
sce_dfs = pd.read_excel("./SCE2022.xlsx", sheet_name=None)
sources_dnf = pd.read_csv("./sources.csv")

In [32]:
ahci_journals = ahci_dfs['wos-core_AHCI 2022-June-22']["Column1"].tolist()
ahci_journals.pop(0)
ahci_journals = [x.lower() for x in ahci_journals]


In [33]:
sce_journals = sce_dfs['June 22, 2022']["Column1"].tolist()
sce_journals.pop(0)
sce_journals = [x.lower() for x in sce_journals]

In [34]:
ssci_journals = ssci_dfs['wos-core_SSCI 2022-June-22']["Column1"].tolist()
ssci_journals.pop(0)
ssci_journals = [x.lower() for x in ssci_journals]

In [35]:
sources_display_name = [x.lower() for x in sources_dnf.display_name.unique()]
sources_alternate_titles = [x.lower() for x in sources_dnf.alternate_titles.unique() if type(x) == str]
sources_alternate_titles = [x.replace("[", "").replace("]", "") for x in sources_alternate_titles]
final_source = np.concatenate((sources_display_name, sources_alternate_titles), axis=None)

In [36]:
def filter_journals(journal_list):
    filtered_journals = []
    for journal in journal_list:
        for source in final_source:
            if journal in source and source not in filtered_journals:
                filtered_journals.append(source)
    return filtered_journals

def percentage(part, whole):
    res = 100 * float(part)/float(whole)
    return str(res) + "%"

In [37]:
ssci_in_sources = filter_journals(ssci_journals)
ahci_in_sources = filter_journals(ahci_journals)
sce_in_sources = filter_journals(sce_journals)

In [38]:
%store ssci_in_sources
%store ahci_in_sources
%store sce_in_sources
%store sources_dnf

Stored 'ssci_in_sources' (list)
Stored 'ahci_in_sources' (list)
Stored 'sce_in_sources' (list)
Stored 'sources_dnf' (DataFrame)
